In [ ]:
from efficientnet.tfkeras import EfficientNetB0
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
for dirname, _, filenames in os.walk(r'C:\Users\VIKAS\Desktop\Final Major Project'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

sns.palplot(colors_dark)
sns.palplot(colors_green)
sns.palplot(colors_red)

In [3]:
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

In [ ]:
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join(r'C:\Users\VIKAS\Desktop\Final Major Project\Brain-Tumor-Classification-DataSet-master (3) (2)\Brain-Tumor-Classification-DataSet-master\Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
for i in labels:
    folderPath = os.path.join(r'C:\Users\VIKAS\Desktop\Final Major Project\Brain-Tumor-Classification-DataSet-master (3) (2)\Brain-Tumor-Classification-DataSet-master\Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
k=0
fig, ax = plt.subplots(1,4,figsize=(20,20))
fig.text(s='Sample Image From Each Label',size=18,fontweight='bold',
             fontname='monospace',color=colors_dark[1],y=0.62,x=0.4,alpha=0.8)
for i in labels:
    j=0
    while True :
        if y_train[j]==i:
            ax[k].imshow(X_train[j])
            ax[k].set_title(y_train[j])
            ax[k].axis('off')
            k+=1
            break
        j+=1

In [6]:
X_train, y_train = shuffle(X_train,y_train, random_state=101)

In [ ]:
X_train.shape

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)

In [ ]:
# Create a mapping of class names to numerical indices
label_to_index = {label: index for index, label in enumerate(labels)}

# Convert string labels to numeric indices for y_train
y_train_new = [label_to_index[label] for label in y_train]

# Convert to a NumPy array
y_train = np.array(y_train_new)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train)

# Convert string labels to numeric indices for y_test
y_test_new = [label_to_index[label] for label in y_test]

# Convert to a NumPy array
y_test = np.array(y_test_new)

# One-hot encode the labels
y_test = tf.keras.utils.to_categorical(y_test)

# Check the shape and dtype for both y_train and y_test
print(f"y_train shape: {y_train.shape}")  # Should be (number_of_samples, number_of_classes)
print(f"y_train dtype: {y_train.dtype}")  # Should be float32

print(f"y_test shape: {y_test.shape}")  # Should be (number_of_samples, number_of_classes)
print(f"y_test dtype: {y_test.dtype}")  # Should be float32


In [10]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [11]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [ ]:
model.summary()

In [13]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [14]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.keras",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [ ]:
print(f"y_train shape: {y_train.shape}")
print(f"y_train dtype: {y_train.dtype}")


In [ ]:


# ... [previous code up to model.fit()]
history = model.fit(X_train, y_train, validation_split=0.1, epochs=1, verbose=1, batch_size=32,
                    callbacks=[tensorboard, checkpoint, reduce_lr])

# Save the model after training
model.save("final_model.keras")  # Saving the final model


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

# Ignore warnings
filterwarnings('ignore')

# Assuming `history` is defined and contains the training history
epochs = [i for i in range(len(history.history['accuracy']))]

# Set up the figure and axes
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

# Extract data from the history object
train_acc = history.history.get('accuracy', [])
train_loss = history.history.get('loss', [])
val_acc = history.history.get('val_accuracy', [])
val_loss = history.history.get('val_loss', [])

# Main title
fig.suptitle('Epochs vs. Training and Validation Accuracy/Loss', size=18, fontweight='bold', fontname='monospace', color='black', alpha=0.8)

# Plot Training and Validation Accuracy
sns.despine()
ax[0].plot(epochs, train_acc, marker='o', markerfacecolor='green', color='lightgreen', label='Training Accuracy')
ax[0].plot(epochs, val_acc, marker='o', markerfacecolor='red', color='lightcoral', label='Validation Accuracy')
ax[0].set_title('Accuracy', fontsize=16)
ax[0].legend(frameon=False)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

# Plot Training and Validation Loss
sns.despine()
ax[1].plot(epochs, train_loss, marker='o', markerfacecolor='green', color='lightgreen', label='Training Loss')
ax[1].plot(epochs, val_loss, marker='o', markerfacecolor='red', color='lightcoral', label='Validation Loss')
ax[1].set_title('Loss', fontsize=16)
ax[1].legend(frameon=False)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')

# Show the figure
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the title
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Make predictions
pred = model.predict(X_test)

# Check the shape of predictions
if pred.shape[1] > 1:  # Multi-class classification
    pred = np.argmax(pred, axis=1)  # Get the predicted class indices
else:  # Binary classification
    pred = (pred > 0.5).astype(int)  # Threshold for binary classification

# Convert the true labels from one-hot encoding (if applicable)
y_test_new = np.argmax(y_test, axis=1)

# Ensure the lengths of predictions and true labels match
if len(pred) != len(y_test_new):
    raise ValueError(f"Length of predictions ({len(pred)}) does not match length of true labels ({len(y_test_new)})")

# Evaluate predictions
accuracy = accuracy_score(y_test_new, pred)
conf_matrix = confusion_matrix(y_test_new, pred)
class_report = classification_report(y_test_new, pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions
pred = model.predict(X_test)

# Check the shape of predictions
if pred.shape[1] > 1:  # Multi-class classification
    pred = np.argmax(pred, axis=1)  # Get the predicted class indices
else:  # Binary classification
    pred = (pred > 0.5).astype(int)  # Threshold for binary classification

# Convert the true labels from one-hot encoding (if applicable)
y_test_new = np.argmax(y_test, axis=1)

# Print classification report
report = classification_report(y_test_new, pred)
print(report)

# Optional: Visualize the confusion matrix
conf_matrix = confusion_matrix(y_test_new, pred)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=np.unique(y_test_new), 
            yticklabels=np.unique(y_test_new))
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_new, pred)

# Create a heatmap for the confusion matrix
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
sns.heatmap(conf_matrix, ax=ax, xticklabels=labels, yticklabels=labels, annot=True,
            cmap=colors_green[::-1], alpha=0.7, linewidths=2, linecolor=colors_dark[3])

# Set title and labels
fig.text(s='Heatmap of the Confusion Matrix', size=18, fontweight='bold',
         fontname='monospace', color=colors_dark[1], y=0.92, x=0.28, alpha=0.8)

# Adjust axis labels for better readability
ax.set_xlabel('Predicted Labels', fontsize=14)
ax.set_ylabel('True Labels', fontsize=14)

# Display the heatmap
plt.show()


In [34]:
import io
import cv2
import numpy as np
from PIL import Image
from ipywidgets import FileUpload, VBox, Button, Output
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model("final_model.keras")  # Adjust the path if necessary

# Create the output widget
output = Output()

# Function to make a prediction on an uploaded image
def img_pred(change):
    try:
        # Check if any files were uploaded
        if uploader.value:
            # For debugging: print the structure of uploader.value
            with output:
                output.clear_output()
                print("uploader.value:", uploader.value)

            # Attempt to get the first uploaded file's content
            uploaded_file_info = list(uploader.value.values())[0]
            img_content = uploaded_file_info['content']  # Access the image content
            
            # Convert image content into an image format
            img = Image.open(io.BytesIO(img_content))
            opencv_image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            img = cv2.resize(opencv_image, (150, 150))
            img = img.reshape(1, 150, 150, 3)

            # Normalize the image (if your model expects normalized input)
            img = img.astype('float32') / 255.0

            # Make prediction
            p = model.predict(img)
            p = np.argmax(p, axis=1)[0]

            # Display prediction result
            with output:
                output.clear_output()
                if p == 0:
                    print('The model predicts: Glioma Tumor')
                elif p == 1:
                    print('The model predicts: No tumor')
                elif p == 2:
                    print('The model predicts: Meningioma Tumor')
                else:
                    print('The model predicts: Pituitary Tumor')
        else:
            with output:
                output.clear_output()
                print("No file uploaded.")
    except Exception as e:
        with output:
            output.clear_output()
            print("Error:", e)

# Create the FileUpload widget
uploader = FileUpload(accept='image/*', multiple=False)

# Create a button to trigger the prediction
predict_button = Button(description='Predict')

# Attach the img_pred function to the predict button's click event
predict_button.on_click(img_pred)

# Display the widgets vertically
display(VBox([uploader, predict_button, output]))


In [42]:
def img_pred(change):
    try:
        # Check if any files were uploaded
        if uploader.value:
            with output:
                output.clear_output()
                print("uploader.value:", uploader.value)

            uploaded_file_info = uploader.value[0]
            img_content = uploaded_file_info['content']
            
            # Convert image content into an image format
            img = Image.open(io.BytesIO(img_content))
            opencv_image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            img = cv2.resize(opencv_image, (150, 150))
            img = img.reshape(1, 150, 150, 3)
            img = img.astype('float32') / 255.0

            # Make prediction
            p = model.predict(img)
            print("Raw prediction output:", p)  # Print the model's raw prediction
            
            # Determine predicted class
            predicted_class = np.argmax(p, axis=1)[0]
            predicted_probability = p[0][predicted_class]

            # Display prediction result
            with output:
                output.clear_output()
                if predicted_class == 0:
                    print('The model predicts: Glioma Tumor (Confidence: {:.2f})'.format(predicted_probability))
                elif predicted_class == 1:
                    print('The model predicts: No Tumor (Confidence: {:.2f})'.format(predicted_probability))
                elif predicted_class == 2:
                    print('The model predicts: Meningioma Tumor (Confidence: {:.2f})'.format(predicted_probability))
                else:
                    print('The model predicts: Pituitary Tumor (Confidence: {:.2f})'.format(predicted_probability))
        else:
            with output:
                output.clear_output()
                print("No file uploaded.")
    except Exception as e:
        with output:
            output.clear_output()
            print("Error:", e)
# Create the FileUpload widget
uploader = FileUpload(accept='image/*', multiple=False)

# Create a button to trigger the prediction
predict_button = Button(description='Predict')

# Attach the img_pred function to the predict button's click event
predict_button.on_click(img_pred)

# Display the widgets vertically
display(VBox([uploader, predict_button, output]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Raw prediction output: [[2.6461351e-20 2.1556251e-32 1.0000000e+00 7.6732739e-21]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Raw prediction output: [[0. 0. 1. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Raw prediction output: [[1.9271937e-19 6.1880440e-33 1.0000000e+00 1.5958471e-20]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Raw prediction output: [[3.4562587e-34 0.0000000e+00 1.0000000e+00 0.0000000e+00]]
